<h1 align=center>A method for determining the location of a new hospital in Greater Toronto area</h1>

## Introduction

With the increase in the population in the Greater Toronto area, the current health centers are increasingly beyond their capacity. Regional authorities and the Ontario Ministry of Health are considering the construction of a new hospital in the region.
In addition to the available space, authorities need to consider population density in order to implement this new infrastructure.

This study will offer an optimal solution based on geographic and demographic information from the Greater Toronto area to find the ideal place to build the new hospital.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Project and data description</a>

2. <a href="#item2">Data exploration</a>

3. <a href="#item3">Analyze locations</a>

4. <a href="#item4">Results</a>

5. <a href="#item5">Conclusion</a>    
</font>
</div>

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

<a id='item1'></a>

## 1. Project and data description

In this project we will use the data set from Statistics Canada, the federal agency established to produce statistics on Canada: 
https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Table.cfm?Lang=Eng&T=304&SR=46&S=87&O=A&RPP=5&PR=35&CMA=0&CSD=0

This dataset from 2016 census will be used to retreive information related to Greater Toronto area and densities of every neighborhoods.

Then, we use the Foursquare API to explore neighborhoods in that area. The _explore function_ will be used to get the existing health centers in each minicipalities. Then the Folium library will help to visualize the neighborhoods and their densities. With the k-means clustering algorithm, we will set up an optimal solution for an additional hospital that is taken account the current hospital locations and the population density.

#### Data exploration

Let's first load the data! From the website, it's not possible to save only data from GTA. We need to extract the usefull data from many files from the entire Ontario data.
We know that the GTA is composed of five Census divisions: City of Toronto, Durham, Halton, Peel and York. These divisions are containing twenty five census subdivisions.


In [3]:
trt_data = pd.read_csv('Toronto_census.CSV')
dur_data = pd.read_csv('Durham_census.CSV')
hal_data = pd.read_csv('Halton_census.CSV')
pel_data = pd.read_csv('Peel_census.CSV')


As the file is containing many unnecessary information, let's clean up data and keep only those related to the project. As 

In [52]:
dur_data.columns

Index(['Geographic code', 'Geographic name', 'Geographic type',
       'Province or territory',
       'Incompletely enumerated Indian reserves and Indian settlements, 2016',
       'Population, 2016', 'Population, 2011', '2011 adjusted population flag',
       'Incompletely enumerated Indian reserves and Indian settlements, 2011',
       '2016 population review flag',
       '2011 population review or received update flag',
       'Population, % change', 'Total private dwellings, 2016',
       'Private dwellings occupied by usual residents, 2016',
       'Land area in square kilometres, 2016',
       'Population density per square kilometre, 2016',
       'National population rank, 2016',
       'Provincial/territorial population rank, 2016'],
      dtype='object')

We choose to use the number of dwellings occupied by usual residents so that hotels are excluded in the dwellings.

Let's first select the data to be used in Peel region!

In [4]:
## Peel region
pel_data.rename(columns={"Geographic name":"Name", "Population, 2016": "2016", "Population, 2011":"2011", "Private dwellings occupied by usual residents, 2016":"Dwellings", "Land area in square kilometres, 2016" : "Area","Population density per square kilometre, 2016": "Density"}, inplace = True)
pel_data=pel_data.loc[:, ['Name','2016','2011', 'Dwellings',"Area","Density"]]#, axis = 1, inplace = True)
peel_cities = ["Brampton","Caledon","Mississauga"]
peel_data=pel_data[pel_data["Name"].isin(peel_cities)]
peel_data

,Name,2016,2011,Dwellings,Area,Density
45,Brampton,593638.0,523906.0,168011.0,266.36,2228.7
46,Caledon,66502.0,59460.0,21256.0,688.16,96.6
47,Mississauga,721599.0,713443.0,240913.0,292.43,2467.6


Now, let's add the Durham region!

In [5]:
#Durham region
dur_data.rename(columns={"Geographic name":"Name", "Population, 2016": "2016", "Population, 2011":"2011", "Private dwellings occupied by usual residents, 2016":"Dwellings", "Land area in square kilometres, 2016" : "Area","Population density per square kilometre, 2016": "Density"}, inplace = True)
dur_data=dur_data.loc[:, ['Name','2016','2011', 'Dwellings',"Area","Density"]]#, axis = 1, inplace = True)
dur_cities = ["Ajax","Brock","Clarington","Oshawa","Pickering","Scugog","Uxbridge","Whitby"]
durh_data=dur_data[dur_data["Name"].isin(dur_cities)]
durh_data

,Name,2016,2011,Dwellings,Area,Density
12,Ajax,119677.0,109600.0,37549.0,67.00,1786.4
13,Brock,11642.0,11341.0,4543.0,423.34,27.5
14,Clarington,92013.0,84548.0,32838.0,611.40,150.5
16,Oshawa,159458.0,149607.0,62595.0,145.64,1094.9
17,Pickering,91771.0,88721.0,30919.0,231.55,396.3
18,Scugog,21617.0,21569.0,8218.0,474.71,45.5
19,Uxbridge,21176.0,20623.0,7663.0,420.95,50.3
20,Whitby,128377.0,122022.0,43529.0,146.66,875.4


Now let's move to Halton region

In [9]:
#Halton region
hal_data.rename(columns={"Geographic name":"Name", "Population, 2016": "2016", "Population, 2011":"2011", "Private dwellings occupied by usual residents, 2016":"Dwellings", "Land area in square kilometres, 2016" : "Area","Population density per square kilometre, 2016": "Density"}, inplace = True)
hal_data=hal_data.loc[:, ['Name','2016','2011', 'Dwellings',"Area","Density"]]#, axis = 1, inplace = True)
hal_cities = ["Burlington","Halton Hills","Milton","Oakville"]
halt_data=hal_data[hal_data["Name"].isin(hal_cities)]
halt_data

,Name,2016,2011,Dwellings,Area,Density
26,Burlington,183314.0,175779.0,71373.0,185.66,987.3
27,Halton Hills,61161.0,59013.0,21078.0,276.27,221.4
28,Milton,110128.0,84362.0,34257.0,363.22,303.2
29,Oakville,193832.0,182520.0,66269.0,138.89,1395.6


Now, it's time for Toronto and York

In [ ]:
#Toronto city and York region
dur_data.rename(columns={"Geographic name":"Name", "Population, 2016": "2016", "Population, 2011":"2011", "Private dwellings occupied by usual residents, 2016":"Dwellings", "Land area in square kilometres, 2016" : "Area","Population density per square kilometre, 2016": "Density"}, inplace = True)
dur_data=dur_data.loc[:, ['Name','2016','2011', 'Dwellings',"Area","Density"]]#, axis = 1, inplace = True)
dur_cities = ["Ajax","Brock","Clarington","Oshawa","Pickering","Scugog","Uxbridge","Whitby"]
durh_data=dur_data[dur_data["Name"].isin(dur_cities)]
durh_data

#### Tranform the data into a *pandas* dataframe

The next task is essentially transforming this data of nested Python dictionaries into a *pandas* dataframe. So let's start by creating an empty dataframe.

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [13]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [14]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [15]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.

In [16]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [17]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [19]:
CLIENT_ID = 'QDXT4GZASN3ZBFYCBB5V41BDYMB5P4MWOKKJ2RFY4ZF3FBFX' # your Foursquare ID
CLIENT_SECRET = '31QLV23QJ5Z1DPUUAQC54CR1KL0KTBNBFF1EZTX0XNFQMSCL' # your Foursquare Secret
VERSION = '20200413' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QDXT4GZASN3ZBFYCBB5V41BDYMB5P4MWOKKJ2RFY4ZF3FBFX
CLIENT_SECRET:31QLV23QJ5Z1DPUUAQC54CR1KL0KTBNBFF1EZTX0XNFQMSCL


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [20]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

Get the neighborhood's latitude and longitude values.

In [21]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [24]:
# type your answer here
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url



'https://api.foursquare.com/v2/venues/search?client_id=QDXT4GZASN3ZBFYCBB5V41BDYMB5P4MWOKKJ2RFY4ZF3FBFX&client_secret=31QLV23QJ5Z1DPUUAQC54CR1KL0KTBNBFF1EZTX0XNFQMSCL&ll=40.7896239,-73.9598939&v=20200413&radius=500&limit=100'

Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 

Send the GET request and examine the resutls

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9524c6df2774001b98607d'},
 'response': {'venues': [{'id': '4a229fa8f964a520797d1fe3',
    'name': 'Jacqueline Kennedy Onassis Reservoir',
    'location': {'address': 'Central Park',
     'crossStreet': 'btwn 85th & 96th St',
     'lat': 40.78451894531291,
     'lng': -73.96096622161959,
     'distance': 575,
     'postalCode': '10024',
     'cc': 'US',
     'neighborhood': 'Central Park',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Central Park (btwn 85th & 96th St)',
      'New York, NY 10024',
      'United States']},
    'categories': [{'id': '56aa371be4b08b9a8d573541',
      'name': 'Reservoir',
      'pluralName': 'Reservoirs',
      'shortName': 'Reservoir',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/lake_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1586832418',
    'hasPerk': False},
   {'id': '4bc27efd461576b04791

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [32]:
results['response']

{'venues': [{'id': '4a229fa8f964a520797d1fe3',
   'name': 'Jacqueline Kennedy Onassis Reservoir',
   'location': {'address': 'Central Park',
    'crossStreet': 'btwn 85th & 96th St',
    'lat': 40.78451894531291,
    'lng': -73.96096622161959,
    'distance': 575,
    'postalCode': '10024',
    'cc': 'US',
    'neighborhood': 'Central Park',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['Central Park (btwn 85th & 96th St)',
     'New York, NY 10024',
     'United States']},
   'categories': [{'id': '56aa371be4b08b9a8d573541',
     'name': 'Reservoir',
     'pluralName': 'Reservoirs',
     'shortName': 'Reservoir',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/lake_',
      'suffix': '.png'},
     'primary': True}],
   'referralId': 'v-1586832418',
   'hasPerk': False},
  {'id': '4bc27efd461576b047917d32',
   'name': 'Central Park - North Meadow Recreation Center',
   'location': {'address': 'Central 

In [39]:
venues = results['response']#['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: "None of [Index(['venue.name', 'venue.categories', 'venue.location.lat',\n       'venue.location.lng'],\n      dtype='object')] are in the [columns]"

And how many venues were returned by Foursquare?

In [37]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

NameError: name 'nearby_venues' is not defined

<a id='item2'></a>

## 2. Explore Neighborhoods in Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [41]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [42]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3161, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [43]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,70,70,70,70,70,70
Carnegie Hill,88,88,88,88,88,88
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,39,39,39,39,39,39
East Village,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [44]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 323 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [45]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaur

And let's examine the new dataframe size.

In [46]:
manhattan_onehot.shape

(3161, 324)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [47]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaur

#### Let's confirm the new size

In [48]:
manhattan_grouped.shape

(40, 324)

#### Let's print each neighborhood along with the top 5 most common venues

In [49]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.09
1          Hotel  0.07
2            Gym  0.06
3  Boat or Ferry  0.04
4  Memorial Site  0.04


----Carnegie Hill----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.06
2  Italian Restaurant  0.03
3              Bakery  0.03
4           Wine Shop  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.04
1   American Restaurant  0.04
2    Chinese Restaurant  0.04
3   Fried Chicken Joint  0.04
4  Gym / Fitness Center  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.09
1          Art Gallery  0.05
2  American Restaurant  0.04
3   Italian Restaurant  0.03
4               Bakery  0.03


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.09
1         Cocktail Bar  0.05
2  American Restaurant  0.04
3               Bakery  0.04
4                  Spa  0.03


----Civic Center----
               venue  freq
0  

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Boat or Ferry,Italian Restaurant,Pizza Place,Playground,Plaza
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Pizza Place,Bakery,Bookstore,Cosmetics Shop,Gym,Gym / Fitness Center,Japanese Restaurant
2,Central Harlem,Gym / Fitness Center,American Restaurant,Seafood Restaurant,Chinese Restaurant,Bar,African Restaurant,Fried Chicken Joint,French Restaurant,Event Space,Library
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Italian Restaurant,Bakery,Cocktail Bar,Pizza Place,Pet Store,Nightclub,Market
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Bakery,Dessert Shop,Spa,Hotpot Restaurant,Vietnamese Restaurant,Optical Shop,Salon / Barbershop


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [52]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 2, 0, 2, 2, 3, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Sandwich Place,Gym,Coffee Shop,Kids Store,Steakhouse,Miscellaneous Shop,Supplement Shop,Shopping Mall,Seafood Restaurant,Tennis Stadium
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Cocktail Bar,American Restaurant,Bakery,Dessert Shop,Spa,Hotpot Restaurant,Vietnamese Restaurant,Optical Shop,Salon / Barbershop
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Deli / Bodega,Coffee Shop,New American Restaurant,Tapas Restaurant,Park
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Lounge,Pizza Place,Café,Restaurant,Park,Bakery,Frozen Yogurt Shop,Pharmacy,Chinese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Cocktail Bar,Liquor Store,Sushi Restaurant,Yoga Studio,Chinese Restaurant


Finally, let's visualize the resulting clusters

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [55]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Bakery,Dessert Shop,Spa,Hotpot Restaurant,Vietnamese Restaurant,Optical Shop,Salon / Barbershop
9,Yorkville,Italian Restaurant,Coffee Shop,Gym,Bar,Deli / Bodega,Pizza Place,Wine Shop,Sushi Restaurant,Diner,Japanese Restaurant
12,Upper West Side,Italian Restaurant,Bar,Bakery,Coffee Shop,Vegetarian / Vegan Restaurant,Pub,Bagel Shop,Sports Bar,Bookstore,Thai Restaurant
19,East Village,Bar,Pizza Place,Coffee Shop,Mexican Restaurant,Wine Bar,Ice Cream Shop,Ramen Restaurant,Speakeasy,Korean Restaurant,Cocktail Bar
20,Lower East Side,Chinese Restaurant,Ramen Restaurant,Art Gallery,Pizza Place,Bakery,Pharmacy,Café,Japanese Restaurant,Park,Coffee Shop
25,Manhattan Valley,Coffee Shop,Mexican Restaurant,Indian Restaurant,Bar,Yoga Studio,Chinese Restaurant,Thai Restaurant,Pizza Place,Caribbean Restaurant,Hawaiian Restaurant
27,Gramercy,Italian Restaurant,American Restaurant,Pizza Place,Bar,Coffee Shop,Cocktail Bar,Playground,Mexican Restaurant,Ice Cream Shop,Grocery Store
33,Midtown South,Korean Restaurant,Hotel,Japanese Restaurant,Dessert Shop,Clothing Store,American Restaurant,Gym / Fitness Center,Bar,Burger Joint,Gym


#### Cluster 2

In [56]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Gym,Coffee Shop,Kids Store,Steakhouse,Miscellaneous Shop,Supplement Shop,Shopping Mall,Seafood Restaurant,Tennis Stadium
15,Midtown,Coffee Shop,Hotel,Sporting Goods Shop,Steakhouse,Clothing Store,Theater,Gym,Spa,Café,Sandwich Place
16,Murray Hill,Sandwich Place,Hotel,Coffee Shop,Sushi Restaurant,Bar,Japanese Restaurant,Gym / Fitness Center,American Restaurant,Burger Joint,Pizza Place
28,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Boat or Ferry,Italian Restaurant,Pizza Place,Playground,Plaza
29,Financial District,Coffee Shop,Pizza Place,Mexican Restaurant,Hotel,Cocktail Bar,Sandwich Place,Juice Bar,American Restaurant,Gym,Park


#### Cluster 3

In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Grocery Store,Chinese Restaurant,Mobile Phone Shop,Deli / Bodega,Coffee Shop,New American Restaurant,Tapas Restaurant,Park
6,Central Harlem,Gym / Fitness Center,American Restaurant,Seafood Restaurant,Chinese Restaurant,Bar,African Restaurant,Fried Chicken Joint,French Restaurant,Event Space,Library
8,Upper East Side,Italian Restaurant,Coffee Shop,Bakery,Exhibit,Spa,French Restaurant,Gym / Fitness Center,Juice Bar,Yoga Studio,Wine Shop
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Cocktail Bar,Gym,Burger Joint,Café,Gym / Fitness Center,Cycle Studio
13,Lincoln Square,Gym / Fitness Center,Plaza,Café,Theater,Performing Arts Venue,Concert Hall,Italian Restaurant,American Restaurant,Wine Shop,Indie Movie Theater
14,Clinton,Theater,Gym / Fitness Center,Coffee Shop,Italian Restaurant,Spa,Wine Shop,Gym,Hotel,American Restaurant,Thai Restaurant
17,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Italian Restaurant,Bakery,Cocktail Bar,Pizza Place,Pet Store,Nightclub,Market
18,Greenwich Village,Italian Restaurant,Café,Coffee Shop,Gym,French Restaurant,Dessert Shop,Sushi Restaurant,Pilates Studio,Indian Restaurant,Gourmet Shop
21,Tribeca,Italian Restaurant,Park,Coffee Shop,American Restaurant,Wine Bar,Spa,Café,Scenic Lookout,Poke Place,Playground
22,Little Italy,Bakery,Café,Chinese Restaurant,Bubble Tea Shop,Mediterranean Restaurant,Hotpot Restaurant,Hotel,Italian Restaurant,Thai Restaurant,Sandwich Place


#### Cluster 4

In [58]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Mexican Restaurant,Lounge,Pizza Place,Café,Restaurant,Park,Bakery,Frozen Yogurt Shop,Pharmacy,Chinese Restaurant
4,Hamilton Heights,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Cocktail Bar,Liquor Store,Sushi Restaurant,Yoga Studio,Chinese Restaurant
5,Manhattanville,Coffee Shop,Chinese Restaurant,Seafood Restaurant,Italian Restaurant,Deli / Bodega,Sushi Restaurant,Park,Mexican Restaurant,Café,Supermarket
7,East Harlem,Mexican Restaurant,Bakery,Thai Restaurant,Deli / Bodega,Latin American Restaurant,French Restaurant,Dance Studio,Beer Bar,Taco Place,Gas Station
11,Roosevelt Island,Park,Pizza Place,Kosher Restaurant,Gym,Greek Restaurant,Coffee Shop,Liquor Store,Dry Cleaner,Sandwich Place,Scenic Lookout
26,Morningside Heights,Park,Coffee Shop,American Restaurant,Bookstore,Burger Joint,Deli / Bodega,Seafood Restaurant,Greek Restaurant,Café,Tennis Court
36,Tudor City,Park,Café,Mexican Restaurant,Greek Restaurant,Diner,Deli / Bodega,Gym / Fitness Center,Dog Run,Coffee Shop,Sushi Restaurant


#### Cluster 5

In [59]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Boat or Ferry,Park,Bar,Baseball Field,Harbor / Marina,Cocktail Bar,Coffee Shop,Heliport,Pet Service,Gym / Fitness Center


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/) and [Polong Lin](https://www.linkedin.com/in/polonglin/). I hope you found this lab interesting and educational. Feel free to contact us if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).